# CNN和ResNet

## 1. 加载相关库

In [2]:
pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/b1/43/28bc858b022f6337326d75f4027d2073aad5432328f01ee1236d847f1b82/torchvision-0.22.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torch==2.7.0 from https://files.pythonhosted.org/packages/aa/3f/85b56f7e2abcfa558c5fbf7b11eb02d78a4a63e6aeee2bbae3bb552abea5/torch-2.7.0-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.10.0 from https://files.pythonhosted.org/packages/8b/54/b1ae86c0973cc6f0210b53d508ca3641fb6d0c56823f288d108bc7ab3cc8/typing_extensions-4.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for sympy>=1.13.3 from https://files.pythonhosted.org/packages/a2/09/77d55d46fd61b4a135c444fc97158ef34a095e5681d0a6c10b75bf356191/sympy-1.14.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
import numpy as np
import matplotlib.pyplot as plt

## 2. 下载和加载MNIST数据集

In [7]:
torchvision.datasets.

SyntaxError: invalid syntax (3968290463.py, line 1)

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

full_train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 划分训练集和验证集（例如：50000 训练 + 10000 验证）
train_size = int(0.83 * len(full_train_dataset))  # 50000
val_size = len(full_train_dataset) - train_size   # 10000
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [9]:
len(full_train_dataset)

60000

### 2.1 可视化单张图像

In [10]:
idx = np.random.randiant(o, len(full_train_dataset))
image, label = mnist_dataset[0]

plt.imshow(image.squeeze(), cmap='gray')
plt.title(f'Label: {label}')
plt.axis('off')
plt.show()


AttributeError: module 'numpy.random' has no attribute 'randiant'

### 2.2 可视化多张图像

In [11]:
import numpy as np

def show_images(dataset, num_images=25):
    plt.figure(figsize=(6, 6))
    for i in range(num_images):
        image, label = dataset[i]
        plt.subplot(5, 5, i + 1)
        plt.imshow(image.squeeze(), cmap='gray')
        plt.title(label, fontsize=8)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

show_images(mnist_dataset)


NameError: name 'mnist_dataset' is not defined

## 3. 构建模型

### 3.1 简单的CNN模型

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 32, 14, 14]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 64, 7, 7]
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [6]:
model = SimpleCNN()
model

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

### 3.2 简化的ResNet

In [82]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.skip = nn.Sequential()

        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.skip(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        return F.relu(out)

class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(16)
        self.layer1 = BasicBlock(16, 32, stride=2)
        self.layer2 = BasicBlock(32, 64, stride=2)
        self.fc = nn.Linear(64 * 7 * 7, 10)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))    # [B, 16, 28, 28]
        x = self.layer1(x)                   # [B, 32, 14, 14]
        x = self.layer2(x)                   # [B, 64, 7, 7]
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
## model = ResNet()
model

## 4. 训练过程

### 4.1 训练函数

In [40]:
def train(model, loader, optimizer, device, epoch, writer):
    model.train()
    running_loss, correct = 0, 0
    total = 0
    for batch_idx, (data, targets) in enumerate(loader):
        data, targets = data.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = F.cross_entropy(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pred = outputs.argmax(dim=1)
        correct += (pred == targets).sum().item()
        total += targets.size(0)

    acc = correct / total
    avg_loss = running_loss / len(loader)
    print(f"Epoch {epoch} - Train Loss: {avg_loss:.4f}, Acc: {acc:.4f}")
    writer.add_scalar("Loss/train", avg_loss, epoch)
    writer.add_scalar("Accuracy/train", acc, epoch)

### 4.2 验证函数

In [102]:
def validate(model, loader, device, epoch, writer):
    model.eval()
    correct = 0
    val_loss = 0
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            val_loss += F.cross_entropy(outputs, targets, reduction='sum').item()
            pred = outputs.argmax(dim=1)
            correct += pred.eq(targets).sum().item()

    val_loss /= len(loader.dataset)
    val_acc = correct / len(loader.dataset)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Accuracy/val", val_acc, epoch)

### 4.3 测试函数

In [111]:
def test(model, loader, device):
    model.eval()
    correct = 0
    test_loss = 0
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            test_loss += F.cross_entropy(outputs, targets, reduction='sum').item()
            pred = outputs.argmax(dim=1)
            correct += pred.eq(targets).sum().item()

    test_loss /= len(loader.dataset)
    test_acc = correct / len(loader.dataset)
    print(f"\n [Test Set] Average loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}")
    return test_loss, test_acc


### 4.3 训练CNN

In [117]:
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
writer = SummaryWriter(log_dir="runs/mnist_CNN")

best_val_acc = 0.0
# 训练多个 epoch
for epoch in range(1, 11):
    train(model, train_loader, optimizer, device, epoch, writer)
    validate(model, val_loader, device, epoch, writer)
    
    # 计算验证准确率
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    val_acc = correct / len(val_loader.dataset)

    # 保存最优模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "CNN_best_model.pth")
        print(f"✅ 模型在 epoch {epoch} 保存，验证准确率: {val_acc:.4f}")

writer.close()

Epoch 1 - Train Loss: 0.1462, Acc: 0.9548
Validation Loss: 0.0568, Accuracy: 0.9821
✅ 模型在 epoch 1 保存，验证准确率: 0.9821
Epoch 2 - Train Loss: 0.0468, Acc: 0.9852
Validation Loss: 0.0546, Accuracy: 0.9817
Epoch 3 - Train Loss: 0.0326, Acc: 0.9898
Validation Loss: 0.0434, Accuracy: 0.9874
✅ 模型在 epoch 3 保存，验证准确率: 0.9874
Epoch 4 - Train Loss: 0.0216, Acc: 0.9930
Validation Loss: 0.0421, Accuracy: 0.9862
Epoch 5 - Train Loss: 0.0164, Acc: 0.9949
Validation Loss: 0.0386, Accuracy: 0.9875
✅ 模型在 epoch 5 保存，验证准确率: 0.9875
Epoch 6 - Train Loss: 0.0134, Acc: 0.9956
Validation Loss: 0.0360, Accuracy: 0.9897
✅ 模型在 epoch 6 保存，验证准确率: 0.9897
Epoch 7 - Train Loss: 0.0115, Acc: 0.9961
Validation Loss: 0.0564, Accuracy: 0.9839
Epoch 8 - Train Loss: 0.0107, Acc: 0.9964
Validation Loss: 0.0441, Accuracy: 0.9894
Epoch 9 - Train Loss: 0.0083, Acc: 0.9972
Validation Loss: 0.0480, Accuracy: 0.9879
Epoch 10 - Train Loss: 0.0066, Acc: 0.9976
Validation Loss: 0.0510, Accuracy: 0.9882


### 4.4 测试CNN

In [125]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
checkpoint = torch.load('CNN_best_model.pth')
model.load_state_dict(checkpoint)

test(model, test_loader, device)



 [Test Set] Average loss: 0.0326, Accuracy: 0.9902


(0.03261887364387512, 0.9902)

### 4.5 训练ResNet

In [127]:
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
writer = SummaryWriter(log_dir="runs/mnist_ResNet")

best_val_acc = 0.0
# 训练多个 epoch
for epoch in range(1, 11):
    train(model, train_loader, optimizer, device, epoch, writer)
    validate(model, val_loader, device, epoch, writer)
    
    # 计算验证准确率
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    val_acc = correct / len(val_loader.dataset)

    # 保存最优模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "ResNet_best_model.pth")
        print(f"✅ 模型在 epoch {epoch} 保存，验证准确率: {val_acc:.4f}")

writer.close()

Epoch 1 - Train Loss: 0.1209, Acc: 0.9627
Validation Loss: 0.0617, Accuracy: 0.9806
✅ 模型在 epoch 1 保存，验证准确率: 0.9806
Epoch 2 - Train Loss: 0.0433, Acc: 0.9863
Validation Loss: 0.0588, Accuracy: 0.9818
✅ 模型在 epoch 2 保存，验证准确率: 0.9818
Epoch 3 - Train Loss: 0.0290, Acc: 0.9907
Validation Loss: 0.0361, Accuracy: 0.9890
✅ 模型在 epoch 3 保存，验证准确率: 0.9890
Epoch 4 - Train Loss: 0.0220, Acc: 0.9926
Validation Loss: 0.0572, Accuracy: 0.9842
Epoch 5 - Train Loss: 0.0167, Acc: 0.9943
Validation Loss: 0.0413, Accuracy: 0.9873
Epoch 6 - Train Loss: 0.0143, Acc: 0.9952
Validation Loss: 0.0435, Accuracy: 0.9896
✅ 模型在 epoch 6 保存，验证准确率: 0.9896
Epoch 7 - Train Loss: 0.0099, Acc: 0.9966
Validation Loss: 0.0449, Accuracy: 0.9881
Epoch 8 - Train Loss: 0.0112, Acc: 0.9963
Validation Loss: 0.0491, Accuracy: 0.9875
Epoch 9 - Train Loss: 0.0087, Acc: 0.9970
Validation Loss: 0.0516, Accuracy: 0.9881
Epoch 10 - Train Loss: 0.0089, Acc: 0.9972
Validation Loss: 0.0494, Accuracy: 0.9872


### 4.6 测试ResNet

In [113]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
checkpoint = torch.load('CNN_best_model.pth')
model.load_state_dict(checkpoint)

test(model, test_loader, device)


 [Test Set] Average loss: 0.0536, Accuracy: 0.9867


(0.05361763286590576, 0.9867)

In [60]:
!pip install --upgrade tensorboard

  Obtaining dependency information for tensorboard from https://files.pythonhosted.org/packages/5d/12/4f70e8e2ba0dbe72ea978429d8530b0333f0ed2140cc571a48802878ef99/tensorboard-2.19.0-py3-none-any.whl.metadata
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 25.1 kB/s eta 0:00:0000:0100:06
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.1
    Uninstalling tensorboard-2.12.1:
      Successfully uninstalled tensorboard-2.12.1


In [115]:
!pip install tensorflow --upgrade --no-deps

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/20/cf/55b68d5896e58e25f41e5bc826c96678073b512be8ca2b1f4b101e0f195c/tensorflow-2.19.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Using cached tensorflow-2.19.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/252.6 MB 3.9 kB/s eta 18:09:14
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/xpzhang/anaconda3/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/Users/xpzhang/anaconda3/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/Users/xpzhang/anaconda3/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "